#### Backward generation(BWD)

1. generate data after integral

In [ ]:
#import random
import time

from backward import generate_bwd

In [ ]:
#from __future__ import division
#from sympy import *
import numpy as np

### Testing ...

In [ ]:
from multiprocessing.pool import ThreadPool

_FINISH = False

start = time.time()
with ThreadPool(processes=4) as p:
    out = p.map(generate_bwd, [100]*10)
    time.sleep(40)
    _FINISH = True
    p.terminate()
    p.join()

print("Finished: ", time.time()-start)    

In [ ]:
len(out)

In [ ]:
file = open(r"./data/bwd_sample1000_0.txt", "w+")
for i in range(len(out)):
    sequence =out[i]
    file.writelines(sequence) 

In [ ]:
filenames = []
for i in range(5):
    filename = "./data/bwd_sample1000_" + str(i) + ".txt"
    filenames.append(filename)
with open(r"./data/bwd_sample5000.txt", 'w+') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())